In [11]:
postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not

In [12]:
#创建词典
def createVocabList(dataset):
    vocabList=set([])
    for line in dataset:
        vocabList=vocabList|set(line)
    return list(vocabList)

In [13]:
#将文本转换成向量
def Words2Vec(dataset,vocablist):
    Matrix=[]
    if type(dataset[0]).__name__=='list':        
        txtnum=len(dataset)
        for i in range(txtnum):
            Vec=[0]*len(vocablist)
            for word in dataset[i]:
                if word in vocablist:
                    Vec[vocablist.index(word)]=1           
            Matrix.append(Vec)
    else:
        Vec=[0]*len(vocablist)
        for word in dataset:
            if word in vocablist:
                Vec[vocablist.index(word)]=1
        Matrix=Vec
    return Matrix

In [14]:
vocablist=createVocabList(postingList)
print vocablist

['cute', 'love', 'help', 'garbage', 'quit', 'I', 'problems', 'is', 'park', 'stop', 'flea', 'dalmation', 'licks', 'food', 'not', 'him', 'buying', 'posting', 'has', 'worthless', 'ate', 'to', 'maybe', 'please', 'dog', 'how', 'stupid', 'so', 'take', 'mr', 'steak', 'my']


In [15]:
wordsMatrix=Words2Vec(postingList,vocablist)
print wordsMatrix
print len(wordsMatrix)

[[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]]
6


In [16]:
import numpy as np

#朴素贝叶斯分类器
def NaiveBayes(Matrix,classList):
    sampleNum=len(Matrix)
    wordNum=len(Matrix[0])
    #计算p(c)
    pc0=classList.count(0)/float(len(classList))
    pc1=classList.count(1)/float(len(classList))
    #计算p(w|c)
    pwc0=np.ones((1,wordNum))
    pwc1=np.ones((1,wordNum))
    ep=2.0
    for row in range(len(Matrix)):
        if classList[row]==0:
            pwc0=pwc0+Matrix[row]
        if classList[row]==1:
            pwc1=pwc1+Matrix[row]
    pwc0=np.log(pwc0/(ep+np.sum(pwc0)))
    pwc1=np.log(pwc1/(ep+np.sum(pwc1)))
    return pwc0,pwc1,pc0,pc1

In [17]:
pwc0,pwc1,pc0,pc1=NaiveBayes(wordsMatrix,classVec)
print pc0,pc1

0.5 0.5


In [18]:
#朴素贝叶斯分类函数
def clssifyNB(inputVec,pwc0,pwc1,pc0,pc1):
    p0=np.sum(np.multiply(inputVec,pwc0))+pc0
    p1=np.sum(np.multiply(inputVec,pwc1))+pc1
    return 0 if p0>p1 else 1

In [19]:
#测试贝叶斯分类器

def predictNB(wordList,vocablist,pwc0,pwc1,pc0,pc1):
    wordsVec=np.array(Words2Vec(wordList,vocablist))
    result=clssifyNB(wordsVec,pwc0,pwc1,pc0,pc1)
    return result

In [20]:
wordList=['stupid', 'garbage','love']
print str(wordList)+ u'属于'+ str(predictNB(wordList,vocablist,pwc0,pwc1,pc0,pc1))+u'类！'

['stupid', 'garbage', 'love']属于1类！


In [21]:
#将文本转换成词袋
def bagofWords2Vec(dataset,vocablist):
    txtnum=len(dataset)
    Matrix=[]
    for i in range(txtnum):
        Vec=[0]*len(vocablist)
        for word in dataset[i]:
            if word in vocablist:
                Vec[vocablist.index(word)]+=1           
        Matrix.append(Vec)
    return Matrix

In [22]:
# -*- coding: utf-8 -*-
import csv
import jieba
import codecs
import re

def read_tingyongci():
    path = "123.txt"   #停用词分析
    stop_words = []
    csv_reader = codecs.open(path,"r", "gbk")
    for row in csv_reader:
        row = row.strip( '\r\n' ).encode("utf-8")
        stop_words.append(row)
    return stop_words
stop_words = read_tingyongci()

# 将utf8的列表转换成unicode
def changeListCode(b):
    a = []
    for i in b:
            try:
                i1 = i.decode('GBK').encode('utf-8')
                a.append(i.decode('GBK').encode('utf-8'))
            except:
                a.append(i)
    return a

def str2num(a):
    b=[]
    for item in a:
        b.append(int(item))
    return b

def readtrain():
    with open('train.csv', 'rb') as csvfile:
        reader = csv.reader(csvfile)
        column1 = [row for row in reader]
    content_train = [i[0] for i in column1[0:]] #第一列为文本内容，并去除列名
    content_train = changeListCode(content_train)
    opinion_train = [i[1] for i in column1[0:]] #第二列为类别，并去除列名
    print '训练集有 %s 条句子' % len(content_train)
    train = [content_train, opinion_train]
    return train

def readexam():
    with open('exam.csv', 'rb') as csvfile:
        reader = csv.reader(csvfile)
        column1 = [row for row in reader]
    content_exam = [i[0] for i in column1[0:]] #第一列为文本内容，并去除列名
    print '测试集有 %s 条句子' % len(content_exam)
    return content_exam

# 对列表进行分词并用空格连接
def segmentWord(cont):
    c = []
    patt1 = re.compile(r'\[.*?\]')
    for i in cont:
        if i != '':
            s=[]
            line1 = re.sub(patt1, '', i)
            wordList = jieba.cut(line1)
            for i in wordList:
                if i >= u'\u4e00' and i <= u'\u9fa5':
                    if i.encode('utf8') in stop_words:
                        pass
                    else:
                        s.append(i.encode('utf8'))
            b = " ".join(s)
            c.append(b)
    return c

#将文档转换成词列表
def words2list(words):
    wordsList=[]
    for line in words:
        wordsList.append(line.split(' '))
    return wordsList

In [23]:
stopwords = read_tingyongci()
train = readtrain()
data_exam = readexam()
exam = changeListCode(data_exam)
content = segmentWord(train[0])
content=words2list(content)
opinion = str2num(train[1])
exam_content = segmentWord(exam)
exam_content=words2list(exam_content)
exam_option=[0]*len(exam_content)
vocablist=createVocabList(content)
trainMatrix=Words2Vec(content,vocablist)
pwc0,pwc1,pc0,pc1=NaiveBayes(trainMatrix,opinion)
for i in range(len(exam_content)):
    if exam_content[i]=='':
        exam_option[i]=0
        continue
    exam_option[i]=predictNB(exam_content[i],vocablist,pwc0,pwc1,pc0,pc1)
csvfile = file('result_nb.csv', 'wb')
writer = csv.writer(csvfile)
for i in range(0,exam.__len__()):
    writer.writerow([exam_option[i],data_exam[i]])

训练集有 2000 条句子
测试集有 171019 条句子


Building prefix dict from the default dictionary ...
Loading model from cache c:\users\dsh\appdata\local\temp\jieba.cache
Loading model cost 0.310 seconds.
Prefix dict has been built succesfully.


KeyboardInterrupt: 